<a href="https://colab.research.google.com/github/tejasmanchi/NLP/blob/main/Fake_News_Classifier(Spacy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install gensim

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
#import gensim.downloader as api
#wv=api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/DS/fake_or_real_news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
import spacy
nlp=spacy.load("en_core_web_lg")

In [ ]:
def preprocess_n_vectorize(text):
    doc = nlp(text)

    filtertok=[]
    for token in doc:
      if token.is_punct or token.is_stop:
        continue
      filtertok.append(token.lemma_)
    return wv.get_mean_vector(filtertok)

In [ ]:
df['vectors']=df['text'].apply(lambda x: preprocess_n_vectorize(x))

In [ ]:
df['label'].replace({'FAKE':0,'REAL':1},inplace=True)

In [ ]:
df.head()

,Unnamed: 0,title,text,label,vectors
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0,"[0.023568358, 0.04508389, 0.004456678, -0.0060..."
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0,"[0.015628273, 0.02790191, 0.0066652526, -0.004..."
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1,"[0.012990256, 0.044699218, -0.0015547309, -0.0..."
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0,"[0.014579854, 0.049130455, -0.0005607041, 0.00..."
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1,"[-0.0030369656, 0.07012432, 0.05126716, -0.009..."


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['vectors'],df['label'],test_size=0.3,random_state=2022)

In [ ]:
x_train2d=np.stack(x_train)
x_test2d=np.stack(x_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(x_train2d,y_train)
y_pred=rfc.predict(x_test2d)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.86      0.82       926
           1       0.85      0.78      0.81       975

    accuracy                           0.82      1901
   macro avg       0.82      0.82      0.82      1901
weighted avg       0.82      0.82      0.82      1901

